<a href="https://colab.research.google.com/github/VVdovichev/Data_analysis_algorithms/blob/main/Lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import numpy as np

__1.__ *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [86]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred + 1e-10) + (1.0 - y) * np.log(1.0 - y_pred + 1e-10))   # err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

__2.__ Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [87]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [88]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [89]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [90]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [91]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [92]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

In [93]:
%%time
W = eval_model(X_st, y, 1000000, eta=42)

0 [-15.43753488 -61.07505588  -0.50228904 -22.25511343] 1.1785958326048056
100000 [-339.64608165  -23.00462262 -101.43438714  258.08884032] 0.0011028035825572879
200000 [-372.88750222  -25.08116808 -111.50950831  283.01780618] 0.0006567308339684708
300000 [-394.68060454  -26.44279421 -118.11454453  299.36174193] 0.0004674100662757863
400000 [-410.91758861  -27.45736099 -123.03555866  311.53898752] 0.0003627640862354831
500000 [-423.86219187  -28.26624345 -126.95869987  321.24712758] 0.0002963825403319133
600000 [-434.62652269  -28.93890689 -130.22104244  329.32015991] 0.00025052713088110213
700000 [-443.83994514  -29.51466729 -133.01333947  336.23006876] 0.00021695440369417195
800000 [-451.89343348  -30.01795034 -135.4540892   342.27006436] 0.0001913132851129421
900000 [-459.0465632   -30.46497366 -137.62196412  347.63481649] 0.00017109050036263302
CPU times: user 40.8 s, sys: 2.36 s, total: 43.2 s
Wall time: 40.4 s


__3.__ Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [94]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(X @ W)
    return y_pred_proba

In [95]:
calc_pred_proba(W, X_st)

array([7.73201897e-04, 7.71471809e-29, 1.00000000e+00, 5.83999329e-70,
       9.99959556e-01, 6.19521235e-28, 1.00000000e+00, 7.46939896e-06,
       9.99274347e-01, 1.00000000e+00])

__4.__ Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [96]:
def calc_pred(W, X):
    y_pred = np.round(sigmoid(X @ W))
    return y_pred


y_pred = calc_pred(W, X_st)

In [97]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

__5.__ * Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [98]:
def TP(y, y_pred):
    return np.sum((y == 1) & (y_pred == 1)) 


def TN(y_, y_pred):
    return np.sum((y == 0) & (y_pred == 0)) 


def FP(y, y_pred):
    return np.sum((y == 0) & (y_pred == 1)) 


def FN(y, y_pred):
    return np.sum((y == 1) & (y_pred == 0)) 


def accuracy(y, y_pred):
    tp = TP(y, y_pred)
    tn = TP(y, y_pred)
    fp = FP(y, y_pred) 
    fn = FN(y, y_pred)
    return (tp + tn) / (tp + tn + fp + fn)



def confusion_matrix(y, y_pred): 
    return np.array([[TN(y, y_pred), FP(y, y_pred)],
                    [FN(y, y_pred), TP(y, y_pred)]])
    


def precision(y, y_pred):
    tp = TP(y, y_pred)
    fp = FP(y, y_pred) 
    return tp / (tp + fp)


def recall(y, y_pred):
    tp = TP(y, y_pred)
    fn = FN(y, y_pred)
    return tp / (tp + fn)
      

def f1_score(y, y_pred):
    return (2 * precision * recall) / (precision + recall)